# 🚀 Aula Aberta de DQN
> Ensinando um módulo a pousar na Lua com redes neurais!

> Você pode checar os slides da aula [aqui](#todo)

## Importando as bibliotecas necessárias

In [1]:
!pip install Box2D # Necessário para nossos ambientes
!pip install gym[box2d] # Ambientes

In [33]:
import gym
from collections import deque
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

Você pode ver a implentação do replay buffer [aqui](./ReplayBuffer.py)

In [4]:
from ReplayBuffer import ReplayBuffer

## Conhecendo nosso ambiente

In [20]:
env = gym.make("LunarLander-v2")

print(f"Formato das observações do nosso agente: {env.observation_space.shape} | Uma possível observação: {env.observation_space.sample()}")
print(f"Número de possíveis ações: {env.action_space.n} | Uma possível ação: {env.action_space.sample()}")

env.close()

Formato das observações do nosso agente: (8,) | Uma possível observação: [ 0.8091425  -1.135604    2.359145   -0.7932247  -1.6031862   0.41732824
 -0.77799165 -0.60098314]
Número de possíveis ações: 4 | Uma possível ação: 1


Como estão configuradas as recompensas:
  - Se a nave pousar ela recebe uma recompensa de $+100$
  - Cada perna que entra em contato com o solo o agente recebe $+10$
  - Acionar as engines faz com que ele receba uma penalidade de $-0.3$ por frame
  - O estado terminal ocorre quando ou o agente morre, recebendo uma penalidade de $-100$ ou quando acaba o tempo da simulação, recebendo mais uma recompensa de $+100$ se estava pousado no alvo.

### Criando um agente aleatório
Só para ver se está tudo funcionando corretamente

In [13]:
def random_env():

    env = gym.make("LunarLander-v2")

    state = env.reset()

    done = False

    while not done:

        action = env.action_space.sample()

        next_state, reward, done, _ = env.step(action)

        env.render()

        state = next_state

    env.close() 

In [15]:
random_env()

## Rede Neural

In [18]:
class LinearNetwork(nn.Module):
    """
    Cria uma rede neural para DQN
    """
    def __init__(self, in_dim, out_dim):
        """
        Inicializa a rede
        
        Parâmetros
        ----------
        in_dim: int
        Dimensão de entrada da rede, ou seja, o shape do estado do ambiente
        
        out_dim: int
        Número de ações do agente neste ambiente
        
        Retorna
        -------
        None
        """
        super(LinearNetwork, self).__init__()

        self.layers = nn.Sequential(
            nn.Linear(in_dim, 128), 
            nn.ReLU(),
            nn.Linear(128, 128), 
            nn.ReLU(), 
            nn.Linear(128, out_dim)
        )

    def forward(self, x):
        """
        Propaga uma entrada pela rede
        """
        return self.layers(x)

## Criando nosso agente

In [31]:
class DQNagent:
    """
    Uma classe que cria um agente DQN que utiliza ReplayBuffer como memória
    """
    def __init__(self, 
                 observation_space, 
                 action_space, 
                 lr=3e-4, 
                 gamma=0.99, 
                 max_memory=100000,
                 epsilon_init=0.5,
                 epsilon_decay=0.995,
                 epsilon_min=0.01,
                 epochs=1):
      
        """
        Inicializa o agente com os parâmetros dados
        
        Parâmetros
        ----------
        
        observation_space: gym.spaces
        O espaço de observação do gym
         
        action_space: gym.spaces
        O espaço de ações do agente modelado no gym
        
        lr: floar, default=3e-4
        A taxa de aprendizado do agente
        
        gamma: float, default=0.99
        O fator de desconto. Se perto de 1. as recompensas futuras terão grande importância,
        se perto de 0. as recompensas mais instantâneas terão maior importância
        
        max_memory: int, default=100000
        O número máximo de transições armazenadas no buffer de memória
        
        epsilon_init: float, default=0.5
        O epsilon inicial do agente. Se próximo de 1. o agente tomará muitas ações
        aleatórias, se proóximo de 0. o agente escolherá as ações com maior
        Q-valor
        
        epsilon_decay: float, default=0.9995
        A taxa de decaimento do epsilon do agente. A cada treinamento o agente tende
        a escolher meno ações aleatórias se epsilon_decay<1
        
        min_epsilon: float, default=0.01
        O menor epsilon possível
        
        
        network: str, default='linear'
        O tipo de rede a ser utilizada para o agente DQN. Por padrão é usada uma rede linear, mas
        pode ser usada uma rede convolucional se o parâmetro for 'conv'
        
        Retorna
        -------
        None
        """

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using: {self.device}")
        self.gamma = gamma
        self.memory = ReplayBuffer(max_memory, observation_space.shape[0])
        self.action_space = action_space
        self.epochs = epochs

        self.epsilon = epsilon_init
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min

        self.dqn = LinearNetwork(observation_space.shape[0], action_space.n).to(self.device)
        self.optmizer = optim.Adam(self.dqn.parameters(), lr=lr)

    def act(self, state):
        """
        Método para o agente escolher uma ação
        
        Parâmetros
        ----------
        
        state
        O estado do agente
        
        Retorna
        -------
        
        action
        A ação escolhida pelo agente
        """

        if np.random.random() < self.epsilon:
            action = self.action_space.sample()
        else:
            with torch.no_grad():
                state = torch.FloatTensor(state).to(self.device)
                action = self.dqn.forward(state).argmax(dim=-1)
                action = action.cpu().numpy()

        return action

    def eps_decay(self):
        """
        Método para aplicar o decaimento do epsilon
        """

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def remember(self, state, action, reward, next_state, done):
        """
        Método para armazenar uma sequência estado, ação, recompensa, próximo estado e done
        no buffer de memória
        """

        self.memory.update(state, action, rewards, next_state, done)

    def train(self, batch_size, save=False):
        """
        Método para treinar o agente
        """
        if batch_size * 10 > self.memory.size:
            return

        for epoch in range(self.epochs):
            states, actions, rewards, next_states, dones = self.memory.sample(batch_size)

            states = torch.as_tensor(states).to(self.device)
            actions = torch.as_tensor(actions).to(self.device).unsqueeze(-1) # Unsqueeze adiciona uma dimensão "1" no índice indicado
            rewards = torch.as_tensor(rewards).to(self.device).unsqueeze(-1) # no caso do -1, ele adiciona na última dimensão
            next_state = torch.as_tensor(next_states).to(self.device)
            dones = torch.as_tensors(dones).to(self.device).unsqueeze(-1)

            q = self.dqn.forward(states).gather(-1, actions.long()) # Pega-se os q-valores das ações do batch

            with torch.no_grad():   # Utilizamos o no_grad pois esse q vamos usar para a loss, não precisa dos gradientes
                q2 = self.dqn.forward(next_states).max(dim=-1, keepdim=True)[0]

                target = (rewards + (1 - dones) * self.gamma * q2).to(self.device)

            loss = F.mse_loss(q, target)
            self.optimizer.zero_grad()
            loss.backward()

        if save:
            self.save_model()

    def save_model(self, model_file):
        torch.save(self.dqn.state_dict(), model_file)
        print(f"\n Model saved: {model_file}")

    def load_model(self, model_file):
        self.dqn.load_state_dict(torch.load(model_file))
        print(f"Model loaded: {model_file}")

In [25]:
def train(agent, env, timesteps, batch_size, render=False):
    total_reward = 0
    episode_returns = deque(maxlen=20)
    avg_returns = []
    episode = 0
    state = env.reset()

    for timestep in range(1, timesteps + 1):
        action = agent.act(state)

        # Tomar a ação escolhida
        next_state, reward, done, info = env.step(action)

        # Guardar as informações geradas pela ação
        agent.remember(state, action, reward, next_state, done)

        # Treinar a rede com base no ReplayBuffer
        agent.train(batch_size, False)

        # Soma as recompensas
        total_reward += reward

        if done:
            episode_returns.append(total_reward)
            episode += 1
            next_state = env.reset()

        agent.eps_decay()

        if episode_returns:
            avg_returns.append(np.mean(episode_returns))

        total_reward *= 1 - done
        ratio = math.ceil(100 * timestep / timesteps)
        avg_return = avg_returns[-1] if avg_returns else np.nan

        # Atualiza o estado
        state = next_state

        if render:
            # Mostra o ambiente
            env.render()

        print(
            f"\r[{ratio:3d}%]",
            f"timestep = {timestep}/{timesteps}",
            f"episode = {episode:3d}",
            f"avg_return = {avg_return:10.4f}",
            f"eps = {agent.epsilon:.4f}",
            sep=", ",
            end="")

    print()
    env.close()
    return avg_returns

In [34]:
BATCH_SIZE = 256
GAMMA = 0.99
EPS_INIT = 1
EPS_END = 0.001
EPS_DECAY = 0.99995
MAX_MEMORY = 1_000_000
TIMESTEPS = 150_000
EPOCHS = 1

env_name = 'LunarLander-v2'
env = gym.make(env_name)
OBS_SPACE = env.observation_space
ACT_SPACE = env.action_space

print("\nTraining DQN")
dqn_net = DQNagent(observation_space=OBS_SPACE,
                    action_space=ACT_SPACE,
                    lr=3e-4,
                    gamma=GAMMA,
                    max_memory=MAX_MEMORY,
                    epsilon_init=EPS_INIT,
                    epsilon_decay=EPS_DECAY,
                    epsilon_min=EPS_END,
                    epochs=EPOCHS)


results_dqn = train(dqn_net, env, TIMESTEPS, BATCH_SIZE, render=False)


Training DQN
Using: cpu


NameError: name 'rewars' is not defined